In [1]:
import os
import json
import requests
import time
import MySQLdb as mysql
import myloginpath

In [2]:
conf = myloginpath.parse("PS")

In [3]:
myloginpath.parse("PS", path="/root/.mylogin.cnf")

{'user': 'capstone2',
 'password': 'cse4187-01',
 'host': '49.247.26.236',
 'port': 3306}

In [2]:
Host = "49.247.26.236"
User = "capstone2"
Password='cse4187-01'
Database="PS"

In [3]:
db=mysql.connect(host=Host,user=User,
                  passwd=Password,db=Database)

In [4]:
query_string = "SELECT * FROM user LIMIT 30"
with db.cursor(cursorclass=mysql.cursors.DictCursor) as cur:
    cur.execute(query_string)
    print(cur.fetchall())

({'id': 1621, 'handle': '0_o'}, {'id': 28848, 'handle': '00_00'}, {'id': 37703, 'handle': '00_jw'}, {'id': 13227, 'handle': '0000064'}, {'id': 11617, 'handle': '0004poppop'}, {'id': 11570, 'handle': '0021'}, {'id': 32768, 'handle': '007hwan'}, {'id': 9469, 'handle': '00cwooh'}, {'id': 29036, 'handle': '00mari'}, {'id': 28747, 'handle': '00mkr'}, {'id': 25715, 'handle': '00osos'}, {'id': 38891, 'handle': '00skgun'}, {'id': 22965, 'handle': '010101'}, {'id': 25842, 'handle': '0101son'}, {'id': 2927, 'handle': '01026488131'}, {'id': 2642, 'handle': '0106658lee'}, {'id': 5229, 'handle': '01089292615'}, {'id': 9935, 'handle': '010psj'}, {'id': 37910, 'handle': '0114ajs00'}, {'id': 27776, 'handle': '01179hk'}, {'id': 25583, 'handle': '01234'}, {'id': 38892, 'handle': '0124sung'}, {'id': 31901, 'handle': '0129benny'}, {'id': 30769, 'handle': '012dmsco'}, {'id': 37419, 'handle': '01825'}, {'id': 788, 'handle': '01jm'}, {'id': 30413, 'handle': '01newbie'}, {'id': 14987, 'handle': '01ppwater'}, 

In [42]:
def call_file_sorted(path):
    directory = os.fsencode(path)
    ret = []
    for file in os.listdir(directory):
        filename = os.fsdecode(file)
        filename = os.path.join(path, filename)
        ret.append(filename)
    ret.sort()
    return ret

def change_path(target_path, source_path, ext):
    basename = os.path.basename(source_path)
    basename = os.path.splitext(basename)[0]
    return_path = os.path.join(target_path, basename) + ext
    return return_path

In [51]:
user_history_dir = "solved_data/user_solved"
user_history_list = call_file_sorted(user_history_dir)

In [82]:
cnt = 0
query_string = "INSERT INTO ac VALUES (%s, %s)"
for user in user_history_list:
    with open(user, "r", encoding="utf-8") as f:
        r = json.load(f)
        handle = r["handle"]
        problems = r["problems"]
        problem_cnt = r["count"]
    assert len(problems) == problem_cnt
    with db.cursor(cursorclass=mysql.cursors.DictCursor) as cur:
        cur.execute("SELECT id from user where handle=\"%s\"" %handle)
        user_id = cur.fetchone()['id']
    arguments = []
    for problem_id in problems:
        if problem_id == -1:
            break
        with db.cursor(cursorclass=mysql.cursors.DictCursor) as cur:
            cur.execute("SELECT * FROM problem WHERE id = %d" %problem_id)
            if cur.fetchone() == None:
                print(problem_id)
                assert False
        arguments.append((user_id, problem_id))
        cnt += 1
        '''
        with db.cursor(cursorclass=mysql.cursors.DictCursor) as cur:
            cur.execute("INSERT INTO ac VALUES (%d, %d);" %(user_id, problem_id))
            db.commit()
            cnt += 1
        '''
    if len(arguments) == 0:
        continue
    with db.cursor(cursorclass=mysql.cursors.DictCursor) as cur:
        cur.executemany(query_string, arguments)
        db.commit()
    if user_id % 100 == 0:
        print(user_id)
    if (cnt // 10) % 100 == 0:
        print(cnt)

100
200
300
548006
400
500
600
700
800
900
996008
1000
1025003
1100
1064000
1065007
1200
1300
1215005
1400
1500
1292007
1310006
1600
1700
1800
1446005
1900
2000
2100
1557000
2200
2300
1641007
2400
2500
2600
1763003
2700
1790009
1797004
2800
1819001
1848005
2900
1862002
3000
1893004
1905003
1914009
1918000
3100
1930005
1937006
3200
3300
2005003
3400
2043009
3500
3600
2104007
3700
2124004
2140001
3800
2166004
3900
2191007
4000
2210005
2218009
2221005
4100
2255003
4200
4300
2286009
4400
2334000
4500
2344009
2352008
4600
2381007
2389007
4700
2415000
4800
4900
5000
2475002
2482001
5100
5200
2537000
5300
5400
2578008
5500
2594005
2613000
5600
5700
2659007
5800
5900
2692000
2703009
6000
2719001
6100
6200
2754007
6300
2785000
2787002
2790006
2795004
6400
6500
2825008
6600
2858004
2861004
6700
6800
2898003
6900
2910002
2926003
7000
7100
2954002
7200
2969001
2975003
2977007
2985003
2988006
7300
2998004
7400
3014000
3018000
7500
3043003
7600
3060005
7700
3067005
7800
3101004
7900
8000
3125009
810

In [83]:
problem_directory = "solved_data/problems"
problem_list = call_file_sorted(problem_directory)

In [89]:
cnt = 0
query_string = "INSERT INTO problem_tag VALUES (%s, %s)"
for problem in problem_list:
    with open(problem, "r", encoding="utf-8") as f:
        res = json.load(f)
        problem_id = res["problemId"]
        tag_list = res["tags"]
    arguments = []
    for tag in tag_list:
        tag_name = tag["key"]
        with db.cursor(cursorclass=mysql.cursors.DictCursor) as cur:
            cur.execute("SELECT id FROM tag WHERE name=\"%s\"" %tag_name)
            tag_id = cur.fetchone()['id']
        arguments.append((problem_id, tag_id))
        cnt += 1
    if len(arguments) == 0:
        continue
    with db.cursor(cursorclass=mysql.cursors.DictCursor) as cur:
        cur.executemany(query_string, arguments)
        db.commit()
    if problem_id % 100 == 0:
        print(problem_id)

1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2200
2300
2500
2600
2800
2900
3000
3100
3300
3700
4000
4500
4600
4800
4900
5000
5100
5600
5800
5900
6000
6200
6300
6400
6500
6600
6700
6800
7600
7800
7900
8200
8300
8900
9300
9400
10000
10100
10600
10700
10800
10900
11000
11400
11500
11600
11800
11900
12000
12100
12400
12700
13100
13200
13300
13400
13600
13700
13900
14000
14400
14500
14600
14700
14800
15000
15500
15600
15700
15900
16000
16100
16200
16300
16400
16500
16600
16700
16900
17200
17300
17400
17500
17600
17800
18000
18100
18200
18300
18500
18600
18800
18900
19000
19200
19600
19700
20000
20200
20300
20500
21000
21100
21200
21600
22200
22900
23000
